In [1]:

    var sparkConf = new SparkConf().setAppName("L-BFGS Example");
    var sc = new SparkContext(sparkConf);

var path =  "../data/mllib/sample_libsvm_data.txt";
    var data = MLUtils.loadLibSVMFile(sc, path);
    var ret = {};

    var numFeatures = data.take(1)[0].getFeatures().size();


 Split initial RDD into two... [60% training data, 40% testing data].


In [2]:
    var trainingInit = data.sample(false, 0.6, 11);
    var test = data.subtract(trainingInit);


 Append 1 into the training data as intercept.


In [3]:
    var training = data.map(function (lp) {
        return new Tuple(lp.getLabel(), MLUtils.appendBias(lp.getFeatures()));
    });

    training.cache();


 Run training algorithm to build the model.


In [4]:
    var numCorrections = 10;
    var convergenceTol = 0.0001;
    var maxNumIterations = 20;
    var regParam = 0.1;
    var w = [];
    for (var i = 0; i < numFeatures + 1; i++) {
        w.push(0.0);
    }
    var initialWeightsWithIntercept = Vectors.dense(w);


In [5]:
    var result = LBFGS.runLBFGS(


 training.rdd(),


In [6]:
        training,
        new LogisticGradient(),
        new SquaredL2Updater(),
        numCorrections,
        convergenceTol,
        maxNumIterations,
        regParam,
        initialWeightsWithIntercept);

    var weightsWithIntercept = result[0];
    ret.loss = result[1];


In [7]:
    var arrayWeightsWithIntercept = weightsWithIntercept.toArray();

    var copyOfWeightsWithIntercept = [];
    for (var ii = 0; ii < arrayWeightsWithIntercept.length - 1; ii++) {
        copyOfWeightsWithIntercept.push(arrayWeightsWithIntercept[ii]);
    }


In [8]:
    var model = new LogisticRegressionModel(Vectors.dense(copyOfWeightsWithIntercept), copyOfWeightsWithIntercept.length);


 Clear the default threshold.


In [9]:
    model.clearThreshold();

    var scoreAndLabels = test.map(function (lp, model) {
        return new Tuple(model.predict(lp.getFeatures()), lp.getLabel());
    }, [model]);


 Get evaluation metrics.


In [10]:
    var metrics = new BinaryClassificationMetrics(scoreAndLabels);
    ret.auROC = metrics.areaUnderROC();

var result = ret;
    print("Loss of each step in training process");
    result.loss.forEach(function (l) {
        print(l);
    });
    print("Area under ROC = " + result.auROC);

    sc.stop();
